In [1]:
%load_ext dotenv
%dotenv

### basic chain structure
- LLM chain: 
    - define prompt
    - define llm
    - define chain
    - run chain.predict()
- Router chain:
    - given different expert chains, decide which expert chain, then return the results
    - define multiple prompts
    - define multiple llm/embeddings
    - define chain (multiple chains at the same time)
    - run predict
- Sequential chain
    - the output from previous step becomes the input for the next step
    - define multiple prompts
    - define multiple llm/embeddings
    - define chain (multiple chains sequentially)
    - run predict
- Transformation chain
    - text processing chain to normalize the inputs

### chain application examples (typical usage structure)
- Document chains
    - used specifically for long text processing (such as summary, vectorDB)
    - Stuff
        - for a given question, pass the question and context together to the LLM
        - however, in many cases, it would exceed the upper limit of the prompt length for LLMs
    - Refine
        - for a given question, pass the question and sliced context to the LLM and give an intermediate answer
        - then, the intermediate answer is passed togehther with question and sliced context to the LLM
        - in this way, it would help to avoid the upper prompt length limit for LLMs
    - Map reduce
    - Map rank
- Retrieval QA

### LLM chain

In [2]:
from langchain.llms import OpenAI
llm = OpenAI()

In [3]:
from langchain import PromptTemplate, LLMChain

prompt_template = "give me a name for a cute Siamese cat"

llm_chain = LLMChain(
    llm = llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

llm_chain.predict()

'\n\n"Koko" '

### Router Chain

In [4]:
naming_template= '''
You are a pet store owner in Japan. 
You are good at naming pets with good Japanese names.

here is the question {input}
'''

math_template = '''
You are a good mathematician. You are great at answering math probrams.
You are so good because you are able to break down hard problems into their component parts, 
answer the component parts, and then put them together to answer the broader question

here is the question {input}
'''

prompt_infos = [
    {
        "name": "naming",
        "description": "good for making Japanese name for pet",
        "prompt_template": naming_template
    },
    {
        "name": "math",
        "description": "good for answering math questions",
        "prompt_template": math_template
    }
]

In [5]:
from langchain.chains import ConversationChain
from langchain.chains.router import MultiPromptChain

In [6]:
# initialize inidividual chain
destination_chains = {}

for p_info in prompt_infos:
    name = p_info['name']
    prompt_template = p_info['prompt_template']
    prompt = PromptTemplate(
        template=prompt_template, input_variables=['input']
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

default_chain = ConversationChain(llm=llm, output_key='text')

In [7]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [9]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

router_prompt = PromptTemplate(
    template = router_template,
    input_variables=['input'],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [10]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

In [11]:
# it first find the naming chain
# then run the chain in the naming branch
print(chain.run("give me a name for a cute Siamese cat"))



> Entering new MultiPromptChain chain...


c:\Users\billl\miniconda3\envs\llm_proj\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


naming: {'input': 'give me a name for a cute Siamese cat'}
> Finished chain.

"Kojiro" (小次郎)


In [12]:
print(chain.run("""
                if A and B are 100mile away from each other and are traveling twowards each other.
                A travels at 50mile per hour, while B travels at 25miles per hour.
                How long does it take A and B to meet each other?
                """))



> Entering new MultiPromptChain chain...


c:\Users\billl\miniconda3\envs\llm_proj\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


math: {'input': '\n    if A and B are 100mile away from each other and are traveling twowards each other.\n    A travels at 50mile per hour, while B travels at 25miles per hour.\n    How long does it take A and B to meet each other?'}
> Finished chain.

To solve this problem, we can use the formula d = rt, where d is the distance, r is the rate, and t is the time. Since A and B are traveling towards each other, their distances will add up to 100 miles.

So, for A, d = 50t and for B, d = 25t. Setting these two equations equal to each other, we get 50t = 25t, or t = 2 hours.

Therefore, it will take A and B 2 hours to meet each other. This is because in 2 hours, A would have traveled 50 miles and B would have traveled 25 miles, adding up to 100 miles and meeting at the middle point.


### Sequential Chain

In [13]:
template= '''
You are a pet store owner in Japan. 
You are good at naming pets with good Japanese names.
Please give three names

here is the question {input}
'''

prompt_template = PromptTemplate(template=template, input_variables=["input"])
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

In [14]:
template= '''
You are a pet name evaluator
You are good at analyzing pet names and decide which one is better

Pet name analysis:
{synopsis}
Analyze the above names and decide which one is better
'''

prompt_template = PromptTemplate(template=template, input_variables=["synopsis"])
review_chain = LLMChain(llm=llm, prompt=prompt_template)

In [15]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(
    chains = [synopsis_chain, review_chain],
    verbose=True
)

In [16]:
review = overall_chain.run("give me a name for a cute Siamese cat")



> Entering new SimpleSequentialChain chain...

1. Sakura (meaning "cherry blossom")
2. Hikaru (meaning "shining" or "radiant")
3. Kaida (meaning "little dragon")

Based on meaning and uniqueness, I would say Sakura is the better pet name. It has a beautiful and meaningful association with the cherry blossom, and it is not a very common pet name. Hikaru is also a nice name, but it is more commonly used in Japan as a human name. Kaida is a cute name, but it may not have as much significance or depth as the other two names. Overall, Sakura would make a great pet name for its beauty and symbolism.

> Finished chain.


### document chain type (long text processing chain)